In [ ]:
%tensorflow_version 1.x

In [ ]:
!cp drive/MyDrive/temp/weights_10_0.02.h5 .

In [ ]:
from __future__ import print_function

import os, cv2, sys
from keras.applications.inception_v3 import InceptionV3
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten, Input
from keras import optimizers
import numpy as np
import glob
import tensorflow as tf

path_to_weights = 'weights_10_0.02.h5'

input_shape = (299, 299, 3)
main_input = Input(shape=input_shape, dtype='float32', name='main_input')
phi_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=main_input, input_shape=input_shape)
phi_features = phi_model.output
phi_flattened = Flatten(name='phi-flattened')(phi_features)
final_output_focal = Dense(1, activation='sigmoid', name='output_focal')(phi_flattened)
final_output_distortion = Dense(1, activation='sigmoid', name='output_distortion')(phi_flattened)

layer_index = 0
for layer in phi_model.layers:
    layer._name = layer.name + "_phi"

model = Model(inputs=main_input, outputs=[final_output_focal, final_output_distortion])
model.load_weights(path_to_weights)
model.compile()
model.summary()

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

!cp model.tflite drive/MyDrive/temp/.

In [ ]:
INPUT_SIZE = 299
image = cv2.imread("./fisheye_00.jpg")
image = cv2.resize(image,(INPUT_SIZE,INPUT_SIZE))
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = image / 255.
image = image - 0.5
image = image * 2.
image = np.expand_dims(image,0)

image = preprocess_input(image) 

prediction_focal = model.predict(image)[0]
prediction_dist = model.predict(image)[1]